##### ***Machine learning in drug discovery - series 2***

<br>

##### **Review of ML series 1 & introduction of series 2**

This post was really a continuation of the first ML series - "Small molecules in ChEMBL database". In particular, I wanted to work on the logistics regression (LR) model, and look into other strategies that I could use to improve it. Last time, I used LogisticRegression() method on the df_ml dataframe (df_ml_pd was the actual dataframe name used in ML series 1, to denote a conversion from a Polars to Pandas dataframe). I've not changed any parameters for the LR estimator, which meant everything was kept at default settings. Overall, this was an example of a default prototype of a LR classifer, which most likely would be too simplistic and not really reflecting real-world equivalents, but it sort of helped me to think in terms of a LR context.

This time, with a goal of trying to improve the model, I've planned to use cross-validation and hyper-parameter tuning at least to evaluate the estimator performance. It was also worth evaluating whether LR was the best ML approach for the df_ml dataset, which would likely need to be kept as a separate post to avoid another lengthy read. I also, on the other hand, have had an idea in mind of doing a ML series 3 looking at re-training the model and a final evaluation, which would keep me busy in the coming weeks or months.

::: callout-note
In *scikit-learn*, an estimator is often referring to the different ML approaches, which are usually grouped into classification (e.g. naive Bayes, logistic regression), regression (e.g. support vector regression), clustering (e.g. K-means clustering) and dimensionality reduction (e.g. principal component analysis). A useful guide to help with choosing the right estimator can be found [here](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
:::

<br>

##### **Import dataframe from ML series 1**

Since *scikit-learn* mainly supports Pandas dataframes for ML, I've opted to use Pandas instead of Polars dataframe library this time, to avoid the extra step of converting a Polars dataframe into a Pandas one.


In [ ]:
import pandas as pd

I've exported the final dataframe from ML series 1 as a .csv file, so that we could continue on this ML series and work on the logistic regression model further. For this ML series 2, the .csv file was imported as shown below.


In [ ]:
df_ml = pd.read_csv("df_ml.csv")
df_ml.head()

In [ ]:
df_ml.shape

<br>

##### **Import libraries for machine learning**


In [ ]:
# Install scikit-learn - an open-source ML library
# Uncomment the line below if needing to install this library
#!pip install -U scikit-learn

In [ ]:
# Import scikit-learn
import sklearn

# Check version of scikit-learn 
print(sklearn.__version__)

Other libraries needed to generate ML model were imported as below.


In [ ]:
# To use NumPy arrays to prepare X & y variables
import numpy as np

# To normalise dataset prior to running ML
from sklearn import preprocessing
# To split dataset into training & testing sets
from sklearn.model_selection import train_test_split

# For data visualisations
# Uncomment line below if requiring to install matplotlib
#!pip install matplotlib
import matplotlib.pyplot as plt

<br>

##### **Logistic regression**

To get the LR model ready, we needed to define our X and y variables from the df_ml dataset.

<br>

###### **Defining X and y variables**


In [ ]:
# Define X variables from df_ml dataset
X = np.asarray(df_ml[["#RO5 Violations", 
                      "QED Weighted", 
                      "CX LogP", 
                      "CX LogD", 
                      "Heavy Atoms"]]
              )
X[0:5]

In [ ]:
# Define y variable
y = np.asarray(df_ml["Max_Phase"])
y[0:5]

<br>

###### **Training and testing sets**


In [ ]:
# Split dataset into training & testing sets

# Random number generator
#rng = np.random.RandomState(0) - note: this may produce different result each time

# Edited post to use random_state = 250 to show comparison with ML series 1
# for reproducible result
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 250)
print('Training set:', X_train.shape, y_train.shape)
print('Testing set:', X_test.shape, y_test.shape)

<br>

###### **Preprocessing data**


In [ ]:
# Normalise & clean the dataset
# Fit on the training set - not on testing set as this might lead to data leakage
# Transform on the testing set
X = preprocessing.StandardScaler().fit(X_train).transform(X_test)
X[0:5]

<br>

###### **Fitting LR classifier on training set**

One major difference to this LR classifier this time was that cross-validation was included, via using the LogisticRegressionCV model, when creating and fitting the training data.


In [ ]:
# Import logistic regression CV estimator
from sklearn.linear_model import LogisticRegressionCV

# Change to LogisticRegressionCV() - LR with built-in cross validation
# Create an instance of logistic regression CV classifier and fit the data
LogR = LogisticRegressionCV().fit(X_train, y_train)
LogR

<br>

###### **Applying LR classifier on testing set for prediction**


In [ ]:
y_mp = LogR.predict(X_test)
y_mp

<br>

###### **Converting predicted values into a dataframe**


In [ ]:
# Predicted values were based on log odds
# Use describe() method to get characteristics of the distribution
pred = pd.DataFrame(LogR.predict_log_proba(X))
pred.describe()

Alternatively, a quicker way to get predicted probabilities was via predict_proba() method in *scikit-learn*.


In [ ]:
y_mp_proba = LogR.predict_proba(X_test)
# Uncomment below to see the predicted probabilities printed
#print(y_mp_proba)

<br>

###### **Converting predicted probabilities into a dataframe**


In [ ]:
# Use describe() to show distributions
y_mp_prob = pd.DataFrame(y_mp_proba)
y_mp_prob.describe()

<br>

###### **Pipeline method for LR**

A quick trial of applying pipeline method with LogisticRegressionCV was done as shown below.


In [ ]:
# Test pipline from scikit-Learn
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

LogReg = make_pipeline(StandardScaler(), LogisticRegressionCV())
LogReg.fit(X_train, y_train)

<br>

##### **Cross-validation & hyper-parameter tuning**

<br>

###### **Cross-validation**

Cross-validation was designed to minimise the amount of samples lost if we were to always partition all of our datasets into three lots for training, testing and validation. Readers with eagle eyes might notice a new set of data named validation set here. One of the biggest reasons to add this validation set was that often overfitting could happen on testing set with testing data being leaked into the model, which most likely would occur because all the parameters could be edited until the model performs optimally. By having a validation set of the data, this overfitting problem would be avoided. 

In general, model training could take place initially on the training set, with the validation set used for first evaluation, which would be followed by a final evaluation on the testing set if the model testing worked as expected. The "cross" part of the cross-validation was the part that described the process of splitting the training data into many smaller number (*k*) of sets, which was also why cross-validation was also often known as "*k*-fold cross-validation" as well. With the use of *k*-fold cross-validation, the training set was essentially equivalent to *k*-1 of the folds of training data. The trained model would then be validated by using the remaining parts of the training data, which was almost like being used as a testing data to measure the performance of the trained model.

::: callout-note
In short, cross-validation was simply a commonly used out-of-sample evaluation metric with effective use of data, where each observation was used for both training and testing.
:::

<br>

###### **Decoding LogisticRegressionCV classifier**

Since we've used LogisticRegressionCV classifier for the LR models, this meant it would be unnecessary to use GridSearchCV again if we read the definition of the estimatorCV closely, as shown below.

As quoted from *scikit-learn* on [cross-validation estimator](https://scikit-learn.org/stable/glossary.html#term-cross-validation-estimator):

<q>An estimator that has built-in cross-validation capabilities to automatically select the best hyper-parameters (see the User Guide). Some example of cross-validation estimators are ElasticNetCV and LogisticRegressionCV. Cross-validation estimators are named EstimatorCV and tend to be roughly equivalent to GridSearchCV(Estimator(), ...). The advantage of using a cross-validation estimator over the canonical estimator class along with grid search is that they can take advantage of warm-starting by reusing precomputed results in the previous steps of the cross-validation process. This generally leads to speed improvements. An exception is the RidgeCV class, which can instead perform efficient Leave-One-Out (LOO) CV. By default, all these estimators, apart from RidgeCV with an LOO-CV, will be refitted on the full training dataset after finding the best combination of hyper-parameters.

Therefore, the cross validation part for the LR model was taken care of by the LogisticRegressionCV() classifier. However, I wanted to find out more about this particular estimator, so to further dissect LogisticRegressionCV classifer, I looked into the documentation in *scikit-learn*, in that Stratified K-Folds cross-validator was used specifically at the default setting. One of the parameters in the classifier that was closely related to the Stratified K-Folds was the cv parameter, which was the cross-validation generator that could be tuned by providing integers as its equivalent number of folds used. Its default value for LogisticRegressionCV was set as "None", which was changed from 3-fold to 5-fold in version 0.22.

<br>

###### **Hyper-parameter tuning - how parameters affect LR model**

To explicitly see the details of all the parameters after cross-validation, we could find the names and values of these parameters for the estimator by using the code below.

To find the parameters of any ML estimator, the code suggested by *scikit-learn* was shown below.

```{{python}}
estimator.get_params()
```

In this example, the first LR model built was named LogR. All the parameters used for LogR could be shown by using the code below.


In [ ]:
LogR.get_params()

The second LR model built using pipeline method was called LogReg, and its parameters used were shown below.


In [ ]:
LogReg.get_params()

Both models acutally, without any surprises, had exactly the same set of parameters when LogisticRegressionCV classifier was used.

However, to find out how the parameters influence the LR model, it was probably best to test a few of the parameters out. I've had two parameters in mind that I thought would affect the confusion matrix at least - cv and random_state parameters after doing some manual trial and errors by changing cv and random_state values in the code. However, upon reading and digging more online information and resources, I quickly realised that cv parameter would probably matter more than random_state parameter. This was based this line from *scikit-learn* on LogisticRegressionCV classifer, 

<q> For the grid of Cs values and l1_ratios values, the best hyperparameter is selected by the cross-validator StratifiedKFold, but it can be changed using the cv parameter. 

So it appeared changing cv parameter could affect Cs and l1_ratios values as well. Also from *scikit-learn* documentation on LR, other parameters that could be tuned were solvers, penalties and C (controls regularisation or penalty strengths, which would be already taken care of if using LogisticRegressionCV classifier). Quite a few other similar online projects using logistic regression in *scikit-learn* had also mentioned that logistic regression in general did not have a lot of critical hyper-parameters to be tuned ([one of the examples](https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/)). Another post I happened to bump into even concluded that better time should be used to link the model results with actual business metrics, rather than trying to use hyper-parameter tuning on LR model. Nevertheless, I still wanted to see how these parameters would affect the LR model in this case, even if it was of minor significance, so that I would fully understand how all of them work together.

Note: only certain penalties and solvers work together.

In order to search and see the variations in the LR parameters, I've opted to use RepeatedStratifiedKFold as the cross-validation method (which was also the default cross-validator method used in LogisticRegressionCV classifier). The "repeat" version would repeat Stratified K-Fold at stated (n) times. Because of this, GridSearchCV would be used to exhaustively search for the best parameters in this case, with the aim to see how the changes in parameters would affect the accuracy scores for each model.


In [ ]:
# Re-sampled y variable so that there were same numbers of samples as X variables
y = np.asarray(df_ml["Max_Phase"].sample(n = 379, random_state = 1))
y.shape

In [ ]:
# **Draft code only - work-in-progress**

# Idea is to show the variations/trends in cv, C, solvers & penalties
# when they are used in combinations to fit and evaluate the models

# Code adapted from: https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
#from sklearn.linear_model import LogisticRegression

# Set up model and parameters to test
model = LogisticRegressionCV()
# Note: default value for cv = 5-fold
cv = [5, 10, 20, 30]
# Note: default value for Cs = 10 (integers or floats only)
Cs = [1, 10, 50, 100]
# Note: default solver = "lbfgs"
solvers = ["lbfgs", "liblinear", "newton-cg", "newton-cholesky"]
#  removed to test
# "sag", "saga" not used as these were faster for large datasets
# (the dataset used in this case was small)
penalty = ["l2"]

# Set up grid search
grid = dict(cv = cv, Cs = Cs, solver = solvers, penalty = penalty)

CV = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 2, random_state = 2)

grid_search = GridSearchCV(estimator = model, param_grid = grid, n_jobs = -1, cv = CV, scoring = "accuracy", error_score = 0)

grid_result = grid_search.fit(X, y)

# Results
print("Best mean test score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_["mean_test_score"]
std = grid_result.cv_results_["std_test_score"]
params = grid_result.cv_results_["params"]
for mean, stdv, param in zip(means, std, params):
    print("%f (%f) with: %r" % (mean, stdv, param))

<br>

#### **Evaluation of the logistic regression CV model**

###### **Accuracy scores**


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_mp, y_test)

The accuracy score was still around 0.7 this time for LogR model, but slightly higher at 0.715 as it was 0.6992084 from ML series 1. The accuracy from the pipeline method (LogReg model) was 0.6939314 this time (which was 0.6965699 from ML series 1).


In [ ]:
LogReg.score(X_test, y_test)

<br>

###### **Confusion matrix**

Again, to visualise the LR classifer built this time using LogisticRegressionCV estimator in a confusion matrix, the same code was used as previously.


In [ ]:
# Import confusion matrix from scikit-learn
from sklearn.metrics import confusion_matrix
# Import itertools - functions to create iterators for efficient looping
import itertools

# Function to print and plot confusion matrix
def plot_confusion_matrix(# Sets a cm object (cm = confusion matrix)
                          cm, 
                          # Sets classes of '1s' (Successes) & '0s' (Non-successes) for the cm
                          classes,
                          # If setting normalize = true, reports in ratios instead of counts
                          normalize,
                          title = 'Confusion matrix',
                          # Choose colour of the cm (using colourmap recognised by matplotlib)
                          cmap = plt.cm.Reds):
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    # Plot the confusion matrix 
    plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation = 45)
    plt.yticks(tick_marks, classes)

    # Floats to be round up to two decimal places if using normalize = True
    # or else use integers
    fmt = '.2f' if normalize else 'd'
    # Sets threshold of 0.5
    thresh = cm.max() / 2.
    # Iterate through the results and differentiate between two text colours 
    # by using the threshold as a cut-off
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment = "center",
                 color = "white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Compute confusion matrix
matrix = confusion_matrix(y_test, y_mp, labels = [0,1])
np.set_printoptions(precision = 2)

# Plot confusion matrix without normalisation
plt.figure()
plot_confusion_matrix(matrix, 
                      # Define classes of outcomes
                      classes = ['Max_Phase = 0','Max_Phase = 1'], 
                      # Set normalize = True if wanting ratios instead
                      normalize = False, 
                      title = "Confusion matrix without normalisation"
                     )

The four different categories in the confusion matrix were:

-   True positive - Predicted Max_Phase = 1 & True Max_Phase = 1 ( out of  samples)
-   True negative - Predicted Max_Phase = 0 & True Max_Phase = 0 ( out of  samples)
-   False positive - Predicted Max_Phase = 1 & True Max_Phase = 0 ( out of  samples)
-   False negative - Predicted Max_Phase = 0 & True Max_Phase = 1 ( out of  samples)

<br>

###### **Classification report**


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_mp))

*Precision* was a measure of the accuracy of a predicted outcome, where a class label had been predicted by the classifier. So in this case, we could see that for class label 1, the precision was 0.72, which corresponded to the true positive result of 128 out of 179 samples (= 0.715). It was defined by:

*Recall*, also known as sensitivity (especially widely used in biostatistics and medical diagnostic fields), was a measure of the strength of the classifier to predict a positive outcome. In simple words, it measured the true positive rate. In this example, there was a total of 128 out of 190 samples (which = 0.674, for True Max_Phase = 1 row) that had a true positive outcome of having a max phase of 1. It was defined by:

The precision and recall metrics could also be calculated for class label = 0, which were shown for the row 0 in the classification report.

*f1-score*, or also known as balanced F-score or F-measure, denoted the harmonic average of both precision and recall metrics. This metric would also give another indication about whether this model performed well on outcome predictions. It normally ranged from 0 (worst precision and recall) to 1 (perfect precision and recall). For this particular classifier, f1-score was at 0.7, which was definitely not at its worst, but also could be further improved. It was defined as:

*Support*, which some readers might have already worked out how the numbers were derived, was the total number of true samples in each class label (read row-wise from the confusion matrix). The main purpose of showing this metric was to help clarifying whether the model or classifier had a reasonably balanced dataset for each class or otherwise.

<br>

###### **Log loss**

Log loss could be used as another gauge to show how good the classifier was at making the outcome predictions. The further the predicted probability was from the true value, the larger the log loss, which was also ranged from 0 to 1. Ideally, the smaller the log loss the better the model would be. Here, we had a log loss of 0.61 for this particular model.


In [ ]:
# Log loss
from sklearn.metrics import log_loss
log_loss(y_test, y_mp_proba)

<br>

#### **Discussions and conclusion**



<br>

#### **Final words**

::: callout-note
Feel free to skip this part as this is really me speaking my thoughts out loud about my situation lately.
:::

After encountering several hiccups during my job hunt in data analytics lately (mainly domestic jobs in NZ), I think I need to make up my mind on which field I'm going for (I might be over-qualified for being a general health data analyst... this was the latest impression/feedback I've received). So here it is, I'm going to concentrate on using machine learning (ML) in Python, specifically in drug discovery field for a while. I'll continue improving the data analytics aspect, but I'll be placing more emphasis on ML so that I can somehow create some ML experiences for myself, while trying to find a good match in the research data science job market in the pharmaceutical field. I also have a feeling that I may need to venture out into overseas roles in the near future, as these types of roles involving ML and pharmaceuticals are just too difficult to spot domestically.

I once read a [blog post on learning ML](https://vickiboykis.com/2022/11/10/how-i-learn-machine-learning/) by V. Boykis, who has suggested to go broadly in topics, then go deep in one of them, which I've agreed wholeheartedly as the approach to go about in the tech world, since there is no way on earth that I will be able to learn everything completely (even OpenAI's ChatGPT has limits - being restricted by the amount and types of input data being fed into the GPT). So since I've branched into 3 programming languages so far, I've decided not to expand further into new programming languages for now, to avoid being "half-bucket-full" for everything, I should really narrow down my focus now. To name the 3 programming languages in the order I've learnt them, they are Python, R and Rust. In that, I'm most comfortable with Python as that is my first language, then it's R, followed by Rust, which is almost brand new. I think right now is a good time for me to go deep into an area that has always caught my attentions.

<br>

#### **References**

-   [scikit-learn documentation](https://scikit-learn.org/stable/index.html)
-   [Scikit-learn: Machine Learning in Python](https://jmlr.csail.mit.edu/papers/v12/pedregosa11a.html), Pedregosa *et al.*, JMLR 12, pp. 2825-2830, 2011.
-   Bruce, P., Bruce, A. & Gedeck P. (2020). Practical statistics for data scientists. O'Reilly.
-   [Stack Overflow](https://stackoverflow.com)